# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
# your code here
import pandas as pd

#Read tic-tac-toe.csv into a dataframe
df = pd.read_csv('./tic-tac-toe.csv')
df

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True
...,...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x,False
954,o,x,o,x,x,o,x,o,x,False
955,o,x,o,x,o,x,x,o,x,False
956,o,x,o,o,x,x,x,o,x,False


In [4]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np


encoder = LabelEncoder()
df_encoded = df.apply(encoder.fit_transform)
df_encoded

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,2,2,2,2,1,1,2,1,1,1
1,2,2,2,2,1,1,1,2,1,1
2,2,2,2,2,1,1,1,1,2,1
3,2,2,2,2,1,1,1,0,0,1
4,2,2,2,2,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
953,1,2,2,2,1,1,1,2,2,0
954,1,2,1,2,2,1,2,1,2,0
955,1,2,1,2,1,2,2,1,2,0
956,1,2,1,1,2,2,2,1,2,0


In [19]:
df_input = df_encoded[['TL','TM','TR','ML','MM','MR','BL','BM','BR']]
df_output = df_encoded['class']


In [27]:
scaler = StandardScaler()
input_normalized = pd.DataFrame(scaler.fit_transform(df_input), columns = df_input.columns)
input_normalized

,TL,TM,TR,ML,MM,MR,BL,BM,BR
0,1.003223,1.084953,1.003223,1.084953,-0.420077,-0.167318,1.003223,-0.167318,-0.286827
1,1.003223,1.084953,1.003223,1.084953,-0.420077,-0.167318,-0.286827,1.084953,-0.286827
2,1.003223,1.084953,1.003223,1.084953,-0.420077,-0.167318,-0.286827,-0.167318,1.003223
3,1.003223,1.084953,1.003223,1.084953,-0.420077,-0.167318,-0.286827,-1.419590,-1.576877
4,1.003223,1.084953,1.003223,1.084953,-0.420077,-0.167318,-1.576877,-0.167318,-1.576877
...,...,...,...,...,...,...,...,...,...
953,-0.286827,1.084953,1.003223,1.084953,-0.420077,-0.167318,-0.286827,1.084953,1.003223
954,-0.286827,1.084953,-0.286827,1.084953,0.930371,-0.167318,1.003223,-0.167318,1.003223
955,-0.286827,1.084953,-0.286827,1.084953,-0.420077,1.084953,1.003223,-0.167318,1.003223
956,-0.286827,1.084953,-0.286827,-0.167318,0.930371,1.084953,1.003223,-0.167318,1.003223


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [28]:
# your code here
X_train, X_test, Y_train, Y_test = train_test_split(input_normalized, df_output, random_state=42)

display(X_train.shape)
display(X_test.shape)
display(Y_train.shape)
display(Y_test.shape)

(718, 9)

(240, 9)

(718,)

(240,)

In [29]:
import tensorflow as tf
import matplotlib.pyplot as plt


model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(2,activation='softmax')
])


model.compile(optimizer='adam',
             los='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=3)

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: inputs=           TL        TM        TR        ML        MM        MR        BL  \
357 -0.286827  1.084953 -1.576877 -0.167318 -1.770525 -0.167318  1.003223   
29   1.003223  1.084953  1.003223 -0.167318 -0.420077 -1.419590 -0.286827   
55   1.003223  1.084953  1.003223 -1.419590 -0.420077  1.084953 -1.576877   
481 -0.286827 -1.419590 -1.576877 -0.167318 -1.770525 -1.419590  1.003223   
331 -0.286827  1.084953 -0.286827 -0.167318  0.930371 -0.167318  1.003223   
..        ...       ...       ...       ...       ...       ...       ...   
106  1.003223  1.084953 -1.576877 -0.167318  0.930371 -0.167318 -0.286827   
270  1.003223 -1.419590 -0.286827 -1.419590  0.930371 -0.167318 -0.286827   
860 -0.286827 -1.419590  1.003223 -0.167318  0.930371 -1.419590 -0.286827   
435 -0.286827 -1.419590  1.003223 -0.167318  0.930371 -0.167318  1.003223   
102  1.003223  1.084953 -1.576877  1.084953 -0.4

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
# your code here

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here